In [2]:
!pip install arxiv


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_core.documents import Document
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=  WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\jasvi\\OneDrive\\Desktop\\sem6\\project_mcd\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://www.kapoorwatch.com/blogs/the-ultimate-collection-8-iconic-luxury-watches-for-the-modern-gentleman/")

docs = loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026EE1A28CD0>)

In [5]:
docs

[Document(page_content=' \n   \n   \n2.1.1  Introduction  \nThe advertising of Coca -Cola has historically been among the most ingenious and creative in \nthe marketing history. They integrated component of mass media used by Coca -Cola includes \ntelevision advertising, banner ads, print advertising, direct marketing, w eb based interactive, \nsales promotion, social media marketing and experience based marketing.  \nIn 2011, Coca Cola completed 125 years. They wanted to celebrate their anniversary by doing \nsomething different that was never tried before along with their coverage of traditional and digital mediums. Obscura Digital, a San Francisco -based technology lab and creative agency \ndesigned and executed something which was really impressive!!!  \nOn May 6, 2011, the Coca -Cola headquarters tower (a 26 -story tall building) in  Atlanta was \nilluminated, showcasing a spectacular vision of the glorious 125 years and a “Thank You” \nmessage for millions of customers who made i

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "watch_ad","Search for information about the watches for any questions asked")

In [13]:

retriever_tool.name

'watch_ad'

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = PyPDFDirectoryLoader("files/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)



In [14]:
documents[45]

Document(page_content='INDUSTr Y AND MArKET  5\nWe were concerned about our major competitor, the Houston Post . \nThey, of course, could decide not to increase their ad or circulation rates \nand gain market share.\nI had been tracking the ad linage each day for the Houston Post  by \npurchasing their newspaper and measuring the ads. I estimated their ad \nrates based on what we learned from customers who bought ads from \nthe Post. Making assumptions based on circulation and the size of their \norganization, I was able to have a pretty good idea of the Post’s revenues, \nexpenses, and contribution margin. My projections showed that the Post \nhad a small contribution margin and my guess was that they too would \nincrease ad rates for two reasons. First, they would increase ad rates to \naccommodate the rise in newsprint costs and second, they would increase \nad rates because local advertisers wanted two major newspapers compet -\ning against each other to hold ad rates down.', metad

In [15]:
db=FAISS.from_documents(documents[:100],OpenAIEmbeddings())
db.save_local("pdf_index")


c:\Users\jasvi\OneDrive\Desktop\sem6\project_mcd\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:
new_db = FAISS.load_local("pdf_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever_pdf = new_db.as_retriever()

retriever_pdf

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020603D42A50>)

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_pdf_tool=create_retriever_tool(retriever, "pdf_search","For any questions about marketing strategies, advertisement startegies refer this")

In [19]:
tools=[wiki,retriever_tool, retriever_pdf_tool]

In [20]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\jasvi\\OneDrive\\Desktop\\sem6\\project_mcd\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='watch_ad', description='Search for information about the watches for any questions asked', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026EDF8C4040>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026EE1A28CD0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000026EDF8C4180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectors

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "YOUR OPENAI SECRET KEY"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [25]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}
agent_scratchpad: {agent_scratchpad}
 """)

In [32]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:

### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [35]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [38]:
agent_executor.invoke({"input":"Tell about BVLGARI: Octo Finissimo"})



> Entering new AgentExecutor chain...

Invoking: `watch_ad` with `{'query': 'BVLGARI Octo Finissimo'}`


BVLGARI: Octo Finissimo


The Octo Finissimo Automatic watch showcases mechanical manufacturing movement with automatic winding. Its stainless steel case, featuring satin-brushed and polished finishes, adds a touch of refinement. The interchangeable bracelet enhances versatility, while the white Clous de Paris dial provides a classic aesthetic. This watch represents the perfect fusion of Italian design, Swiss watchmaking expertise, and jewelry craftsmanship, establishing Bulgari as the Jeweller of Time.
Specifications

Dial: White
Case Material: Stainless Steel
Case Size: 41mm
Movement: Automatic
Functions: Hours, Minutes, Seconds, and Date
Power Reserve: 42 Hours
Water Resistance: 10 Bar (100 Meters)
Glass: Scratch-resistant Sapphire With Double-sided Anti-reflection Treatment
Strap Type: Stainless Steel


CARTIER: Ballon Bleu De Cartier

Designed for jet pilots, the Avenger B01 

{'input': 'Tell about BVLGARI: Octo Finissimo',
 'output': 'The BVLGARI Octo Finissimo is a remarkable timepiece that showcases mechanical manufacturing movement with automatic winding. It features a stainless steel case with satin-brushed and polished finishes, adding a touch of refinement. The watch has a white Clous de Paris dial, providing a classic aesthetic. The Octo Finissimo Automatic watch represents the perfect fusion of Italian design, Swiss watchmaking expertise, and jewelry craftsmanship, establishing Bulgari as the Jeweller of Time.\n\nSpecifications:\n- Dial: White\n- Case Material: Stainless Steel\n- Case Size: 41mm\n- Movement: Automatic\n- Functions: Hours, Minutes, Seconds, and Date\n- Power Reserve: 42 Hours\n- Water Resistance: 10 Bar (100 Meters)\n- Glass: Scratch-resistant Sapphire With Double-sided Anti-reflection Treatment\n- Strap Type: Stainless Steel\n\nIf you need more information or details about the BVLGARI Octo Finissimo, feel free to ask!'}

In [43]:
answer=agent_executor.invoke({"input":"Build a marketing approach to sell BVLGARI Octo Finissimo watch"})



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': 'marketing approach for BVLGARI Octo Finissimo watch'}`


BVLGARI: Octo Finissimo


The Octo Finissimo Automatic watch showcases mechanical manufacturing movement with automatic winding. Its stainless steel case, featuring satin-brushed and polished finishes, adds a touch of refinement. The interchangeable bracelet enhances versatility, while the white Clous de Paris dial provides a classic aesthetic. This watch represents the perfect fusion of Italian design, Swiss watchmaking expertise, and jewelry craftsmanship, establishing Bulgari as the Jeweller of Time.
Specifications

Dial: White
Case Material: Stainless Steel
Case Size: 41mm
Movement: Automatic
Functions: Hours, Minutes, Seconds, and Date
Power Reserve: 42 Hours
Water Resistance: 10 Bar (100 Meters)
Glass: Scratch-resistant Sapphire With Double-sided Anti-reflection Treatment
Strap Type: Stainless Steel


CARTIER: Ballon Bleu De Cartier

Designed f

In [47]:
ans=answer['output']

In [46]:
from langchain.chains import LLMChain
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a detailed prompt to generate an image based on the following description: {image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)

c:\Users\jasvi\OneDrive\Desktop\sem6\project_mcd\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [48]:
from langchain.agents import initialize_agent, load_tools

tools = load_tools(["dalle-image-generator"])
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
output = agent.run(ans)

c:\Users\jasvi\OneDrive\Desktop\sem6\project_mcd\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\jasvi\OneDrive\Desktop\sem6\project_mcd\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Dall-E-Image-Generator to create images that showcase the unique features, quality craftsmanship, and brand heritage of the Octo Finissimo watch.
Action: Dall-E-Image-Generator
Action Input: Image descriptions for the distinctive features, high-quality materials, and brand heritage of the Octo Finissimo watch.
Observation: https://oaidalleapiprodscus.blob.core.windows.net/private/org-QK2iTPriQkDbULILppq6lbd2/user-juL7WnSyAHC8dEgmtpIA3vh7/img-O7p5PrCylBu3yZ9H3LisJW4X.png?st=2024-05-09T16%3A52%3A26Z&se=2024-05-09T18%3A52%3A26Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-05-08T21%3A58%3A25Z&ske=2024-05-09T21%3A58%3A25Z&sks=b&skv=2021-08-06&sig=Rn0RqlyR9Oc/DgtGUoKEIdYG3udBnni3PdNP1HQ3SZc%3D
Thought: I can also use Dall-E-Image-Generator to create images that showcase the versatility and elegance of the Octo Finissimo watch.
Action: Dall

In [40]:
agent_executor.invoke({"input":"What is customer segmenation and how can it be used to sell watches?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Customer segmentation`


Page: Market segmentation
Summary: In marketing, market segmentation is the process of dividing a broad consumer or business market, normally consisting of existing and potential customers, into sub-gCustomer segmentation is the process of dividing a broad consumer market into sub-groups of individuals who have similar characteristics, needs, or behaviors. This segmentation allows businesses to tailor their products, services, and marketing strategies to meet the specific needs of each segment. 

In the context of selling watches, customer segmentation can be used effectively to target different groups of watch buyers based on various factors such as demographics, psychographics, behavior, and preferences. Here's how customer segmentation can be used to sell watches:

1. **Demographic Segmentation**: Watches can be marketed differently to various age groups, genders, income levels, and occupat

{'input': 'What is customer segmenation and how can it be used to sell watches?',
 'output': "Customer segmentation is the process of dividing a broad consumer market into sub-groups of individuals who have similar characteristics, needs, or behaviors. This segmentation allows businesses to tailor their products, services, and marketing strategies to meet the specific needs of each segment. \n\nIn the context of selling watches, customer segmentation can be used effectively to target different groups of watch buyers based on various factors such as demographics, psychographics, behavior, and preferences. Here's how customer segmentation can be used to sell watches:\n\n1. **Demographic Segmentation**: Watches can be marketed differently to various age groups, genders, income levels, and occupations. For example, luxury watches may be targeted towards high-income individuals, while trendy and affordable watches may appeal to younger consumers.\n\n2. **Psychographic Segmentation**: Unders

In [31]:
tools[2]

Tool(name='marketing search', description='Search for information about marketing and advertisement techniques. For any question about making an advertisemnt campaign for any product use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E0FC9CBB00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E0EBC22290>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001E0FC9CBD80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E0EBC22290>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))